<a href="https://colab.research.google.com/github/matteolimoncini/glasses_or_no_glasses/blob/develop/Glasses_or_not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --user kaggle

In [ ]:
#probably useless
!export KAGGLE_USERNAME=matteolimoncini
!export KAGGLE_KEY=c4df437bb6e9108ad29ae9744b503c89

In [ ]:
!mkdir /root/.kaggle
!echo '{"username":"matteolimoncini","key":"c4df437bb6e9108ad29ae9744b503c89"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jeffheaton/glasses-or-no-glasses
!unzip glasses-or-no-glasses.zip -d glasses-or-no-glasses

100% 6.11G/6.11G [02:12<00:00, 39.7MB/s]

